In [12]:
import pandas as pd

A = pd.read_csv("A.csv").values


import numpy as np

x = np.array(A[:, 0:3].astype(np.float32))

y = np.array(A[:, 3:5].astype(np.float32))


from homography import Transformer

t = Transformer(img_size=512)
t.fit_homography(x, y)


import pickle

with open('./transformer.pickle', 'wb') as f:
    pickle.dump(t, f)

Homography Residuals: [0.00242039]
Least Squares Check: b, A @ X, b - A @ X
[[ 1.89453125e-01  1.85412303e-01  4.04082185e-03]
 [ 1.03515625e-01  9.34743404e-02  1.00412846e-02]
 [ 4.60937500e-01  4.49132447e-01  1.18050527e-02]
 [ 4.29687500e-01  4.21947030e-01  7.74047024e-03]
 [ 4.16015625e-01  4.10748497e-01  5.26712799e-03]
 [ 6.26953125e-01  6.10679390e-01  1.62737352e-02]
 [ 8.33984375e-01  8.38516184e-01 -4.53180865e-03]
 [ 8.78906250e-02  9.37170243e-02 -5.82639928e-03]
 [ 4.55078125e-01  4.59626799e-01 -4.54867440e-03]
 [ 8.32031250e-01  8.57598957e-01 -2.55677070e-02]
 [ 4.45312500e-01  4.44487474e-01  8.25025896e-04]
 [ 5.17578125e-01  5.06008184e-01  1.15699411e-02]
 [ 5.21484375e-01  5.24569517e-01 -3.08514181e-03]
 [ 8.32031250e-01  8.15979829e-01  1.60514213e-02]
 [ 4.70703125e-01  4.67775204e-01  2.92792143e-03]
 [ 4.35546875e-01  4.45569965e-01 -1.00230898e-02]
 [ 2.20703125e-01  2.33403450e-01 -1.27003250e-02]
 [ 2.57812500e-01  2.78072156e-01 -2.02596563e-02]]


In [13]:
    with open("./transformer.pickle", 'rb') as f:
        transformer = pickle.load(f)

In [14]:
transformer.x_scale

0.1

In [15]:
transformer.y_scale

0.1

In [16]:
x

array([[-6.1601562, 11.394531 ,  9.465935 ],
       [-0.6418457,  1.1850586,  9.989228 ],
       [-1.3933105, -2.8947754,  9.989228 ],
       [ 5.19311  ,  4.43164  , 15.23828  ],
       [-0.682373 , -6.401855 , 11.99092  ],
       [-0.8232422, -3.3674316, 15.540527 ],
       [ 0.3359375, -6.0234375,  9.98703  ],
       [-0.5180664, -2.2033691, 11.738525 ],
       [-4.9975586,  4.974365 ,  7.9972534]], dtype=float32)

In [17]:
t.homography_transform(x)

array([[0.18008994, 0.08024847],
       [0.42693213, 0.40739048],
       [0.39500483, 0.56203653],
       [0.85522305, 0.11519649],
       [0.51922833, 1.19261404],
       [0.45284778, 0.62325085],
       [0.534528  , 0.7641677 ],
       [0.45358647, 0.49414195],
       [0.2497464 , 0.30414256]])

In [18]:
y

array([[0.18945312, 0.10351562],
       [0.4609375 , 0.4296875 ],
       [0.41601562, 0.6269531 ],
       [0.8339844 , 0.08789062],
       [0.45507812, 0.83203125],
       [0.4453125 , 0.5175781 ],
       [0.5214844 , 0.83203125],
       [0.47070312, 0.43554688],
       [0.22070312, 0.2578125 ]], dtype=float32)